In [9]:
import numpy as np
import pandas as pd
import geopandas as gpd


file = pd.ExcelFile('Wijkdata_Jeugdhulp_in_de_wijk.ods')

sheet_names = file.sheet_names

wijk = pd.read_excel(file, sheet_name=sheet_names[-3])  # wijk
gebr = pd.read_excel(file, sheet_name=sheet_names[-2])  # gebruik
det = pd.read_excel(file, sheet_name=sheet_names[-1])  # determinanten

file.close()

df = det.copy()
# als je wijk en gemeentecode hebt heb je wijkcode en gemeentenaam niet nodig
df = df.merge(wijk[['wijk', 'gemeentecode']], on='wijk', how='left')
# per_jhzv : aandeel jeugdigen met jeugdhulp zonder verblijf, waargenomen -> target variable
df = df.merge(gebr[['wijk', 'perc_jhzv']], on='wijk', how='left')

for col in df.columns:
    df[col] = df[col].apply(lambda x: x if x != '.' else np.nan)

# TODO is nu missig values vullen met gemiddelde hoe willen we dat doen?
df.fillna(df.mean(), inplace=True)

# hun schatting root mean squared error (residu = schatting - waargenomen)
for col in gebr.columns:
    gebr[col] = gebr[col].apply(lambda x: x if x != '.' else np.nan)

# drop missende schattingen en bereken RMSE
original_rmse = str((gebr.dropna(subset='residu')['residu']**2).mean()**0.5)

# buurt_geo = gpd.read_file('./WijkBuurtkaart_2022_v1/buurt_2022_v1.shp')
wijk_geo = gpd.read_file('./WijkBuurtkaart_2022_v1/wijk_2022_v1.shp')
# gemeente_geo = gpd.read_file('./WijkBuurtkaart_2022_v1/gemeente_2022_v1.shp')

wijk_geo = gpd.read_file('./WijkBuurtkaart_2022_v1/wijk_2022_v1.shp')
wijk_geo['WK_CODE'] = wijk_geo['WK_CODE'].apply(lambda x: x[2:]).astype('float')
wijk_geo['GM_CODE'] = wijk_geo['GM_CODE'].apply(lambda x: x[2:]).astype('float')
wijk_final = df.merge(wijk_geo, left_on=['wijk', 'gemeentecode'], right_on=['WK_CODE', 'GM_CODE'], how='left')
for col in wijk_final.columns:
    wijk_final[col] = wijk_final[col].apply(lambda x: x if x != int(-99999999) else np.nan)


df = wijk_final
df = df.drop(['JRSTATCODE', 'JAAR', 'Shape_Leng', 'Shape_Area', 'geometry', 'wijk', 'WK_CODE', 'WK_NAAM', 'GM_CODE', 'GM_NAAM', 'gemeentecode', 'H2O'], axis=1)
train_df = df.drop('perc_jhzv', axis=1)

In [35]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from impyute.imputation.cs import fast_knn, mice
from sklearn.linear_model import TweedieRegressor

# start the KNN training
X_mean = train_df.fillna(train_df.mean())
X_median = train_df.fillna(train_df.median())
X_interpolate = train_df.interpolate(method='linear').fillna(0)
X_knn = fast_knn(train_df.values, k=30)
y = df['perc_jhzv']

X_train, X_test, y_train, y_test = train_test_split(X_median, y, test_size=0.2, random_state=42)

# Instantiate the model
model = LinearRegression()

# Train the model
model.fit(X_train, y_train)

# Predict the test data
y_pred = model.predict(X_test)

# Calculate RMSE of the model
linear_rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("Linear regression RMSE : ", linear_rmse)
print("Original RMSE: ", original_rmse)

Linear regression RMSE :  2.81100313186846
Original RMSE:  2.2551381084753688


In [32]:
# DOCUMENTATIE -----------------------------------------------------------------------------------------------------------------

# Er is geprobeerd de waardes in te vullen door middel van het nemen van het gemiddelde, het nemen van de mediaan (deze
# is overigens hetzelfde als 0 invullen voor elke missende waarde), door middel van interpolatie, of door middel van k-nearest neighbours. 
# De eerste methode brengt een RMSE van 2.8433 bij linear regression, de tweede methode een RMSE van 2.8110, 
# de derde methode een RMSE van 2.8394, en de vierde methode een RMSE van 2.8576